In [ ]:
!pip install lightfm
!pip install pandas
!pip install kaggle
!kaggle datasets download -d rdoume/beerreviews
!unzip beerreviews.zip


Dataset URL: https://www.kaggle.com/datasets/rdoume/beerreviews
License(s): unknown
beerreviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  beerreviews.zip
replace beer_reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from scipy.sparse import coo_matrix
from google.colab import files
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import pandas as pd
import time
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
data = pd.read_csv('beer_reviews.csv')

In [ ]:
data = data.dropna(subset=["review_profilename", "beer_name", "beer_beerid"])
data = data[["review_profilename", "beer_beerid", "beer_name", "review_overall"]]

In [ ]:
unique_users = data['review_profilename'].unique()
num_users = len(unique_users)
user_features = pd.DataFrame({
    'review_profilename': unique_users,
    'age': np.random.randint(18, 65, size=num_users),
    'gender': np.random.choice(['М', 'Ж'], size=num_users),
    'location': np.random.choice(['Москва', 'Питер', 'Казань', 'Белгород'], size=num_users),
})

In [ ]:
data = pd.merge(data, user_features, on='review_profilename')

In [ ]:
le_beer = LabelEncoder()
data['beer_beerid_encoded'] = le_beer.fit_transform(data['beer_beerid'])

In [ ]:
X = data[['age', 'gender', 'location']]
X = pd.get_dummies(X, columns=['gender', 'location'])
y = data['beer_beerid_encoded']


In [ ]:
print(data.head)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=10, warm_start=True)
for epoch in range(1, 11):
    model.n_estimators += 1
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    print(f"Эпоха {epoch}: Точность: {accuracy:.4f}")

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели на тестовых данных: {accuracy:.2f}")

In [ ]:
print("Введите ваши данные:")
try:
    user_age = int(input("Ваш возраст: "))
    user_gender = input("Ваш пол (м/ж): ").strip().lower()
    user_location = input("Ваш город: ").strip()

    # Проверка пола
    if user_gender == 'м':
        user_gender = 'Male'
    elif user_gender == 'ж':
        user_gender = 'Female'
    else:
        print("Ошибка: Некорректный ввод пола. Используйте 'м' или 'ж'.")
        exit()

    # Проверка города
    if user_location not in ['USA', 'Canada', 'UK', 'Germany']:
        print("Ошибка: Город должен быть одним из ['USA', 'Canada', 'UK', 'Germany'].")
        exit()

    # Подготовка данных для предсказания
    user_data = pd.DataFrame([{
        'age': user_age,
        'gender': user_gender,
        'location': user_location,
    }])
    user_data = pd.get_dummies(user_data, columns=['gender', 'location'])
    user_data = user_data.reindex(columns=X.columns, fill_value=0)

    # Предсказание
    predicted_beer_id = model.predict(user_data)[0]
    predicted_beer_name = le_beer.inverse_transform([predicted_beer_id])[0]

        print(f"Мы рекомендуем вам попробовать пиво: {predicted_beer_name}")

except ValueError:
    print("Ошибка: Пожалуйста, введите числовое значение для возраста.")
